In [ ]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import os

In [ ]:
database_configuration = open(os.getcwd()+r'\db.cfg').read().split(' ')
engine = create_engine('mysql+mysqlconnector://{}:{}@{}:{}/{}'.format(*database_configuration))

In [ ]:
articulos_query = open(os.getcwd()+r'\articulos.sql').read()
df_articulos = pd.read_sql(articulos_query, engine)
df_articulos.info()

In [ ]:
# Agrupamos por proveedor.
df_grouped_ref = df_articulos[['AM_COD_PROVEEDOR','PR_NOMBRE_PROVEEDOR','AM_PN']]\
                .groupby(['AM_COD_PROVEEDOR','PR_NOMBRE_PROVEEDOR'],as_index=False)\
                .agg('count')
# Las referencias a 1?????, fallos en importacion.
df_grouped_ref   

In [ ]:
# Agrupamos por proveedor. Aquellos que tengan stock.
df_grouped_stock = df_articulos[df_articulos['AM_STOCK']>0][['AM_COD_PROVEEDOR','PR_NOMBRE_PROVEEDOR','AM_PN']]\
            .groupby(['AM_COD_PROVEEDOR','PR_NOMBRE_PROVEEDOR'],as_index=False)\
            .agg('count')
df_grouped_ref

In [ ]:
# Cruzamos las tablas.
df_referencia_stock = pd.merge(df_grouped_ref, df_grouped_stock, on=['AM_COD_PROVEEDOR'])
df_referencia_stock = df_referencia_stock.drop(columns=['PR_NOMBRE_PROVEEDOR_y'])
df_referencia_stock.columns = ['CODIGO', 'PROVEEDOR', 'REFERENCIAS', 'CON STOCK']
df_referencia_stock.sort_values (['REFERENCIAS'], ascending=False)

In [ ]:
df = df_referencia_stock
df['PORCENTAJE CON STOCK'] = (df['CON STOCK']*100)/df['REFERENCIAS']
df.sort_values (['PORCENTAJE CON STOCK'], ascending=False)

In [ ]:
# Porcentaje medio con stock
df['PORCENTAJE CON STOCK'].mean()

In [ ]:
# Porcentaje medio con stock sin INGRAM MICRO
df = df[df.CODIGO != '000048']
df['PORCENTAJE CON STOCK'].mean()